# SHL Dataset Training and Inference with Dense and CNN Network

Made by STMicroelectronics


Authors: Fabrizio Maria Aymone, Danilo Pau

contact: danilo.pau@st.com

### Download SHL Dataset

In [ ]:
!wget http://www.shl-dataset.org/wp-content/uploads/SHLDataset_preview_v1_part1.zip
!7za x SHLDataset_preview_v1_part1.zip -y

In [ ]:
# path related to the dataset folder

root = './SHLDataset_preview_v1'

# frame size and slide size

window_size = 24
slide = 12

# sensors list
list_sensors = ['label','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']

# user dates
user_dates_1 = ['220617', '260617','270617']

In [ ]:
import pandas as pd

def load_data(position, user, user_dates):
    data_tmp = []
    label_tmp = []
    for i in range(len(user_dates)):
        data_path = root+'/User'+str(user)+'/'+user_dates[i]+'/'+position+'_Motion.txt' 
        label_path = root+'/User'+str(user)+'/'+user_dates[i]+'/Label.txt' 
        lb = pd.read_csv(label_path, sep = " ", header = None)
        df = pd.read_csv(data_path, sep = " ", header = None)
        data_tmp.append(df)
        label_tmp.append(lb)
    data = pd.concat([x for x in data_tmp], ignore_index = True)
    labels = pd.concat([x for x in label_tmp], ignore_index = True)
    return data, labels

In [ ]:
import sklearn
#import imblearn
from sklearn.preprocessing import StandardScaler
import numpy as np

def preprocess(position, user, user_dates):
    
    data, labels = load_data(position, user, user_dates)
    
    df = pd.DataFrame()
    df['acc_x'] = data[1]
    df['acc_y'] = data[2]
    df['acc_z'] = data[3]
    df['gy_x'] = data[4]
    df['gy_y'] = data[5]
    df['gy_z'] = data[6]
    df['activity'] = labels[1]
    df.drop(df.loc[df['activity']==0].index, inplace=True)
    df = df.dropna()

    label = sklearn.preprocessing.LabelEncoder()
    df['label'] = label.fit_transform(df['activity'])
    
    X = df.loc[:,["acc_x", "acc_y", "acc_z", "gy_x", "gy_y", "gy_z"]]
    y = df.loc[:,"label"]
    # over_sampler = imblearn.over_sampling.RandomOverSampler(random_state=42)
    # X, y = over_sampler.fit_resample(df, df['label'])

    #int16 symmetric quantization
    max_acc  = max(np.abs(np.min(X[["acc_x", "acc_y", "acc_z"]])), np.max(X[["acc_x", "acc_y", "acc_z"]]))
    max_gy =  max(np.abs(np.min(X[["gy_x", "gy_y", "gy_z"]])), np.max(X[["gy_x", "gy_y", "gy_z"]]))

    scale_acc = 2*max_acc/(2**(16)-1)
    scale_gy = 2*max_gy/(2**(16)-1)

    X[["acc_x", "acc_y", "acc_z"]] = np.round(X[["acc_x", "acc_y", "acc_z"]]/scale_acc).astype(np.int16)
    X[["gy_x", "gy_y", "gy_z"]] = np.round(X[["gy_x", "gy_y", "gy_z"]]/scale_gy).astype(np.int16)

    df = pd.DataFrame(X)
    df['label'] = y
    
    return df

In [ ]:
import tensorflow as tf
import scipy
from tensorflow import keras
from itertools import chain
import gc

class CustomWindowGenerator(keras.utils.Sequence):

    def __init__(self, X, y, index_array=None, window_size=256, batch_size=64, slide=50, shuffle=True, pass_array=False):
        
        # X and y are lists of datasets e.g. if we have two datasets (X1, y1) and (X2, y2), X is [X1, X2] and y is [y1, y2]
        
        self.X = X
        self.y = y
        self.w = window_size
        self.batch_size = batch_size
        self.slide = slide
        self.shuffle = shuffle
        self.index_array= np.array(list(chain(*[zip(np.repeat(idx, len(x)), np.arange(0, (len(x)-window_size+1), slide)) for idx, x in enumerate(X)])) if pass_array==False else index_array)
        if self.shuffle:
            np.random.shuffle(self.index_array)
        #self.epoch=1

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.index_array)
        gc.collect()
        keras.backend.clear_session()
    
    def __getitem__(self, index):

        X_d = []
        y_d = []
        
        for n, i in self.index_array[index*self.batch_size:index*self.batch_size + self.batch_size]:
            
            X_d.append([self.X[n][i:i+self.w]])
            y_d.append(scipy.stats.mode(self.y[n][i:i+self.w])[0])
        
        return np.vstack(X_d), np.vstack(y_d)
    
    def __len__(self):
        return len(self.index_array)//self.batch_size

# Dense Models

In [ ]:
def get_dense_model(num_classes):

    gc.collect()
    keras.backend.clear_session()
    
    
    model = keras.Sequential([
            keras.layers.Input((24,6)),
            keras.layers.Dense(128),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dense(64),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dense(128),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Flatten(),
            keras.layers.Dense(32),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dense(num_classes, activation = 'softmax'),
            ])

    return model

In [ ]:
from qkeras import *

def get_dense_quantized_model(num_classes):

        gc.collect()
        keras.backend.clear_session()
    
    
        #qkeras model

        quantized_model = keras.Sequential([
                        keras.layers.Input(shape=(24,6)),
                        #QActivation("quantized_bits(16)"),
                        QDense(128, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QDense(64, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QDense(128, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        keras.layers.Flatten(),
                        QDense(32, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QDense(num_classes, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.Activation('softmax'),
                        ])

        return quantized_model

In [ ]:
import os
import sklearn.model_selection
# training hyperparameters

batch_size = 256 
epochs = 100

def train(position, users, folder):

        df = preprocess(position, users, folder) 
        X = df[["acc_x", "acc_y", "acc_z", "gy_x", "gy_y", "gy_z"]].values.astype(np.float32)
        y = df[["label"]].values.astype(np.float32)

        num_classes = len(np.unique(y))
        print("Number of classes: ", num_classes)
        
        one_hot_y = keras.utils.to_categorical(y, num_classes)

        kfold = sklearn.model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8, random_state=1)
        
        fold = 1
        
        results_k = []
        results_tflite = []
        results_qk = []


        index_array = np.array(list(chain(*[zip(np.repeat(idx, len(x)), np.arange(0, (len(x)-window_size+1), slide)) for idx, x in enumerate([X])])))
        
        y_index_array = []
        for _, i in index_array:
                y_index_array.append(scipy.stats.mode(y[i:i+window_size])[0])    
        

        train_index_array, test_index_array = sklearn.model_selection.train_test_split(index_array, test_size=0.2, random_state=42, shuffle=True, stratify=y_index_array)
        
        test_gen = CustomWindowGenerator([X], [one_hot_y], index_array=test_index_array, window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)

        X_test_lite = []
        y_test_lite = []

        for _,i in test_index_array:
                X_test_lite.append(X[i:i+window_size])
                y_test_lite.append(scipy.stats.mode(y[i:i+window_size])[0])
        
        X_test_lite = np.asarray(X_test_lite, dtype=np.float32)
        y_test_lite = np.asarray(y_test_lite, dtype=np.int8)

        # K fold validation 5 steps

        for train_idx, val_idx in kfold.split(X = np.zeros(len(train_index_array)), y = y[train_index_array[:,1]]):
                
                train_gen = CustomWindowGenerator([X], [one_hot_y], index_array=train_index_array[train_idx], window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)
                val_gen = CustomWindowGenerator([X], [one_hot_y], index_array=train_index_array[val_idx], window_size=window_size, batch_size=2048, slide=slide, shuffle=True, pass_array=True)
                
                model = get_dense_model(num_classes)

                model.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])
                
                print("K fold validation step:",fold)

                # model callbacks
                earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

                keras_models_dir = "dense/keras_models/"
                os.makedirs(keras_models_dir, exist_ok=True)
                mcp_path = keras_models_dir+'model_best_'+str(users)+'_'+position+'_k_fold_'+str(fold)+'.h5'
                mcp_save = keras.callbacks.ModelCheckpoint(mcp_path, save_best_only=True, monitor='val_loss', mode='min')
                
                reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
                
                # model training
                
                model.fit(train_gen, validation_data=val_gen, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[earlyStopping, reduce_lr_loss, mcp_save])
                
                model = keras.models.load_model(mcp_path)

                #evaluating the model on test data

                result = model.evaluate(test_gen)
                print("\n\n*****************************************keras model accuracy {} result: {}*****************************************".format(fold, result[1]))
                results_k.append(result[1])






                def representative_data_gen():
                        repr_index_array = np.arange(0, len(X)-window_size+1, slide)
                        np.random.shuffle(repr_index_array)
                        for i in range(100):
                                j = repr_index_array[i]
                                yield [np.expand_dims(X[j:j+window_size], axis=0)]


                # load best keras model to tensorflow liet converter for tflite conversion and quantization
                converter = tf.lite.TFLiteConverter.from_keras_model(model)
                converter.optimizations = [tf.lite.Optimize.DEFAULT]
                converter.representative_dataset = representative_data_gen
                # Ensure that if any ops can't be quantized, the converter throws an error
                converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
                # Set the input and output tensors to float32 and int8 (APIs added in r2.3)
                converter.inference_input_type = tf.float32
                converter.inference_output_type = tf.int8

                tflite_model_quant = converter.convert()

                tflite_models_dir = "dense/tflite_models/"
                os.makedirs(tflite_models_dir, exist_ok=True)
                tflite_model_path = tflite_models_dir+"tflite"+'_'+str(position)+'_'+str(fold)+'.tflite'

                with open(tflite_model_path, 'wb') as f:
                        f.write(tflite_model_quant)
        
                # Input and output details

                tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
                input_details = tflite_interpreter.get_input_details()
                output_details = tflite_interpreter.get_output_details()
                tflite_interpreter.resize_tensor_input(input_details[0]['index'], (X_test_lite.shape))
                tflite_interpreter.allocate_tensors()

                #quantize
                scale, zero_point = input_details[0]["quantization"]

                tflite_interpreter.set_tensor(input_details[0]['index'], X_test_lite)

                tflite_interpreter.invoke()

                scale, zero_point = output_details[0]['quantization']

                tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])

                y_pred = tflite_model_predictions
                y_pred = (tflite_model_predictions.astype(np.int8))
                result = sklearn.metrics.accuracy_score(np.squeeze(y_test_lite), y_pred.argmax(axis=1))
                print("*****************************************TFlite model accuracy score: "+str(result)+"*****************************************\n\n")
                results_tflite.append(result)







                qmodel = get_dense_quantized_model(num_classes)

                qmodel.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])

                # model callbacks
                earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

                qkeras_models_dir = "dense/qkeras_models/"
                os.makedirs(qkeras_models_dir, exist_ok=True)
                mcp_path = qkeras_models_dir+'model_best_'+str(users)+'_'+position+'_k_fold_'+str(fold)+'.h5'
                mcp_save = keras.callbacks.ModelCheckpoint(mcp_path, save_best_only=True, monitor='val_loss', mode='min')
                
                reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
                
                # model training
                
                qmodel.fit(train_gen, validation_data=val_gen, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[earlyStopping, reduce_lr_loss, mcp_save])
                
                qmodel.load_weights(mcp_path)

                #evaluating the model on test data

                result = qmodel.evaluate(test_gen)
                print("\n\n*****************************************qkeras model accuracy {} result: {}*****************************************".format(fold, result[1]))
                results_qk.append(result[1])
                
                fold+=1
        
        results_k, results_tflite, results_qk = np.asarray(results_k), np.asarray(results_tflite), np.asarray(results_qk)
        os.makedirs(f"./dense/results/{position}", exist_ok=True)
        np.save(f"./dense/results/{position}/results_keras.npy", results_k)
        np.save(f"./dense/results/{position}/results_tflite.npy", results_tflite)
        np.save(f"./dense/results/{position}/results_qkeras.npy", results_qk)
        
        return results_k, results_tflite, results_qk

In [ ]:
train("Hand", 1, user_dates_1)
train("Hips", 1, user_dates_1)
train("Torso", 1, user_dates_1)
train("Bag", 1, user_dates_1)

# CNN Models

In [ ]:
def get_cnn_model(num_classes):

    gc.collect()
    keras.backend.clear_session()
    
    
    model = keras.Sequential([
        keras.layers.Input((24,6)),
        keras.layers.Reshape((6,4,6)),
        keras.layers.Conv2D(filters=8, kernel_size = (5,5), padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv2D(filters=8, kernel_size = (5,5), padding= "same"),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPool2D(pool_size = 2),
        keras.layers.Flatten(),
        keras.layers.Dense(64),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Dense(num_classes, activation = 'softmax'),
        ]) 

    return model

In [ ]:
from qkeras import *

def get_cnn_quantized_model(num_classes):

        gc.collect()
        keras.backend.clear_session()
    
    
        #qkeras model

        quantized_model = keras.Sequential([
                        keras.layers.Input((24,6)),
                        keras.layers.Reshape((6,4,6)),
                        #QActivation("quantized_bits(16)"),
                        QConv2D(filters=8, kernel_size = (5,5), padding = "same", kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QConv2D(filters=8, kernel_size = (5,5), padding = "same", kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        keras.layers.MaxPool2D(pool_size = 2),
                        keras.layers.Flatten(),
                        QDense(64, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QDense(num_classes, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.Activation('softmax'),
                        ])

        return quantized_model
    

In [ ]:
import os

# training hyperparameters

batch_size = 64 
epochs = 100

def train(position, users, folder):

        df = preprocess(position, users, folder) 
        X = df[["acc_x", "acc_y", "acc_z", "gy_x", "gy_y", "gy_z"]].values.astype(np.float32)
        y = df[["label"]].values.astype(np.float32)

        num_classes = len(np.unique(y))
        print("Number of classes: ", num_classes)
        
        one_hot_y = keras.utils.to_categorical(y, num_classes)

        kfold = sklearn.model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8, random_state=1)
        
        fold = 1
        
        results_k = []
        results_tflite = []
        results_qk = []


        index_array = np.array(list(chain(*[zip(np.repeat(idx, len(x)), np.arange(0, (len(x)-window_size+1), slide)) for idx, x in enumerate([X])])))
        
        y_index_array = []
        for _, i in index_array:
                y_index_array.append(scipy.stats.mode(y[i:i+window_size])[0])    
        

        train_index_array, test_index_array = sklearn.model_selection.train_test_split(index_array, test_size=0.2, random_state=42, shuffle=True, stratify=y_index_array)
        
        test_gen = CustomWindowGenerator([X], [one_hot_y], index_array=test_index_array, window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)

        X_test_lite = []
        y_test_lite = []

        for _,i in test_index_array:
                X_test_lite.append(X[i:i+window_size])
                y_test_lite.append(scipy.stats.mode(y[i:i+window_size])[0])
        
        X_test_lite = np.asarray(X_test_lite, dtype=np.float32)
        y_test_lite = np.asarray(y_test_lite, dtype=np.int8)

        # K fold validation 5 steps

        for train_idx, val_idx in kfold.split(X = np.zeros(len(train_index_array)), y = y[train_index_array[:,1]]):
                

                train_gen = CustomWindowGenerator([X], [one_hot_y], index_array=train_index_array[train_idx], window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)
                val_gen = CustomWindowGenerator([X], [one_hot_y], index_array=train_index_array[val_idx], window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)
                
                model = get_cnn_model(num_classes)

                model.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])
                
                print("K fold validation step:",fold)

                # model callbacks
                earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

                keras_models_dir = "cnn/keras_models/"
                os.makedirs(keras_models_dir, exist_ok=True)
                mcp_path = keras_models_dir+'model_best_'+str(users)+'_'+position+'_k_fold_'+str(fold)+'.h5'
                mcp_save = keras.callbacks.ModelCheckpoint(mcp_path, save_best_only=True, monitor='val_loss', mode='min')
                
                reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
                
                # model training
                
                model.fit(train_gen, validation_data=val_gen, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[earlyStopping, reduce_lr_loss, mcp_save])
                
                model = keras.models.load_model(mcp_path)

                #evaluating the model on test data

                result = model.evaluate(test_gen)
                print("\n\n*****************************************keras model accuracy {} result: {}*****************************************".format(fold, result[1]))
                results_k.append(result[1])


                        



                def representative_data_gen():
                        repr_index_array = np.arange(0, len(X)-window_size+1, slide)
                        np.random.shuffle(repr_index_array)
                        for i in range(100):
                                j = repr_index_array[i]
                                yield [np.expand_dims(X[j:j+window_size], axis=0)]


                # load best keras model to tensorflow liet converter for tflite conversion and quantization
                converter = tf.lite.TFLiteConverter.from_keras_model(model)
                converter.optimizations = [tf.lite.Optimize.DEFAULT]
                converter.representative_dataset = representative_data_gen
                # Ensure that if any ops can't be quantized, the converter throws an error
                converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
                # Set the input and output tensors to float32 and int8 (APIs added in r2.3)
                converter.inference_input_type = tf.float32
                converter.inference_output_type = tf.int8

                tflite_model_quant = converter.convert()

                tflite_models_dir = "cnn/tflite_models/"
                os.makedirs(tflite_models_dir, exist_ok=True)
                tflite_model_path = tflite_models_dir+"tflite"+'_'+str(position)+'_'+str(fold)+'.tflite'

                with open(tflite_model_path, 'wb') as f:
                        f.write(tflite_model_quant)
        
                # Input and output details

                tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
                input_details = tflite_interpreter.get_input_details()
                output_details = tflite_interpreter.get_output_details()
                tflite_interpreter.resize_tensor_input(input_details[0]['index'], (X_test_lite.shape))
                tflite_interpreter.allocate_tensors()

                #quantize
                scale, zero_point = input_details[0]["quantization"]

                tflite_interpreter.set_tensor(input_details[0]['index'], X_test_lite)

                tflite_interpreter.invoke()

                scale, zero_point = output_details[0]['quantization']

                tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])

                y_pred = tflite_model_predictions
                y_pred = (tflite_model_predictions.astype(np.int8))
                result = sklearn.metrics.accuracy_score(np.squeeze(y_test_lite), y_pred.argmax(axis=1))
                print("*****************************************TFlite model accuracy score: "+str(result)+"*****************************************\n\n")
                results_tflite.append(result)







                qmodel = get_cnn_quantized_model(num_classes)

                qmodel.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])

                # model callbacks
                earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

                qkeras_models_dir = "cnn/qkeras_models/"
                os.makedirs(qkeras_models_dir, exist_ok=True)
                mcp_path = qkeras_models_dir+'model_best_'+str(users)+'_'+position+'_k_fold_'+str(fold)+'.h5'
                mcp_save = keras.callbacks.ModelCheckpoint(mcp_path, save_best_only=True, monitor='val_loss', mode='min')
                
                reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
                
                # model training
                
                qmodel.fit(train_gen, validation_data=val_gen, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[earlyStopping, reduce_lr_loss, mcp_save])
                
                qmodel.load_weights(mcp_path)

                #evaluating the model on test data

                result = qmodel.evaluate(test_gen)
                print("\n\n*****************************************qkeras model accuracy {} result: {}*****************************************".format(fold, result[1]))
                results_qk.append(result[1])
                
                fold+=1
                

        results_k, results_tflite, results_qk = np.asarray(results_k), np.asarray(results_tflite), np.asarray(results_qk)
        os.makedirs(f"./cnn/results/{position}", exist_ok=True)
        np.save(f"./cnn/results/{position}/results_keras.npy", results_k)
        np.save(f"./cnn/results/{position}/results_tflite.npy", results_tflite)
        np.save(f"./cnn/results/{position}/results_qkeras.npy", results_qk)

        return results_k, results_tflite, results_qk

In [ ]:
train("Hand", 1, user_dates_1)
train("Hips", 1, user_dates_1)
train("Torso", 1, user_dates_1)
train("Bag", 1, user_dates_1)

# Final Results

## Dense

In [ ]:
example = "./dense/keras_models/model_best_1_Hand_k_fold_1.h5"
model = keras.models.load_model(example)
model.summary()

In [ ]:
example = "./dense/qkeras_models/model_best_1_Hand_k_fold_1.h5"
qmodel = get_dense_quantized_model(8) #the classes are 8
qmodel.load_weights(example)

In [ ]:
dense_results = {"Hand":[], "Hips":[], "Torso":[], "Bag":[]}

for pos in dense_results.keys():
    dense_results[pos].append([np.load(f"./dense/results/{pos}/results_keras.npy"), np.load(f"./dense/results/{pos}/results_tflite.npy"), np.load(f"./dense/results/{pos}/results_qkeras.npy")])

In [ ]:
from matplotlib import pyplot as plt


from tabulate import tabulate
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import scienceplots

plt.style.use(['science','ieee', 'no-latex'])

positions = ["Hand", "Hips", "Torso", "Bag"]
def plot_metrics(results, title):

    fig, axs = plt.subplots(1, len(positions), figsize=(16, 4))
    fig.subplots_adjust(top=0.8)
    for ax, pos in enumerate(positions):
        keras_acc = np.asarray(results[pos])[0][0]
        tflite_acc = np.asarray(results[pos])[0][1]
        qkeras_acc = np.asarray(results[pos])[0][2]

        x = [0, 1, 2]
        labels = ["Keras", "TFlite", "QKeras"]

        axs[ax].errorbar(x, [np.average(keras_acc), np.average(tflite_acc), np.average(qkeras_acc)], yerr=[np.std(keras_acc), np.std(tflite_acc), np.std(qkeras_acc)], capsize=3, fmt="r--o", ecolor = "black")
        print(np.average(keras_acc), np.average(tflite_acc), np.average(qkeras_acc), np.std(keras_acc), np.std(tflite_acc), np.std(qkeras_acc))
        axs[ax].set_xticks(x, labels, rotation="horizontal")
        axs[ax].set_title(pos)
    fig.suptitle(title, fontsize=16, y=0.95)
    fig.text(0.09, 0.5, 'Accuracy', va='center', rotation='vertical', fontsize=10)
    plt.plot()

In [ ]:
plot_metrics(dense_results, "Dense")

## CNN

In [ ]:
example = "./cnn/keras_models/model_best_1_Hand_k_fold_1.h5"
model = keras.models.load_model(example)
model.summary()

In [ ]:
example = "./cnn/qkeras_models/model_best_1_Hand_k_fold_1.h5"
qmodel = get_cnn_quantized_model(8) #the classes are 8
qmodel.load_weights(example)

In [ ]:
cnn_results = {"Hand":[], "Hips":[], "Torso":[], "Bag":[]}

for pos in cnn_results.keys():
    cnn_results[pos].append([np.load(f"./cnn/results/{pos}/results_keras.npy"), np.load(f"./cnn/results/{pos}/results_tflite.npy"), np.load(f"./cnn/results/{pos}/results_qkeras.npy")])

In [ ]:
plot_metrics(cnn_results, "CNN")